### TODO
1. Ticker pool

### Problem (e.g. point72)
0. Ticker with value: (SIRI)
1. Without Ticker
2. Put none stock
3. Total 1237
4. This holding is removed from the portfolio   in this quarter
5. request time interval
6. float16 can't round
7. some name already have put or call in it.
8. Activity(%) data type
9. null: Activity(%): 501, Ticker: 63
10. Which columns should be index? (Ticker PRIMARY Key can't be null)

### URL rule
https://www.insidermonkey.com/hedge-fund/point72+asset+management/750/holdings/#/ffp=2020-03-31&fot=3&fso=2&pfp=-1&fundType=0
https://www.insidermonkey.com/services/get_fund_holdings.php?hfid=750&module=holdings&fundType=0&ffp=2020-03-31&fot=3&fso=2&offset=40
https://www.insidermonkey.com/services/get_fund_holdings.php?hfid=4&module=holdings&fundType=0&ffp=2020-03-31&fot=5&fso=2&offset=0
https://www.insidermonkey.com/services/get_fund_holdings.php?hfid=4&module=holdings&fundType=0&ffp=2020-03-31&fot=4&fso=1&offset=0

- Prefix:　https://www.insidermonkey.com/
- hfid: Hedge Funds(TODO: need mapping table)
- offset=80: (start at 81)
- ffp=2020-03-31: (2020 Q2)
- fot: which column to sort
- fso=2: ascending, fso=1:descending
- fundType: unknown

### Tips
Tag.find `recursive=False` (without any attribute)

In [1]:
from bs4 import BeautifulSoup, Comment
import requests
# import pickle
import qgrid
import re
import pandas as pd
import numpy as np
import mlcrate as mlc
pd.options.display.max_rows = 100

In [2]:
def get_data():
    semester_list = ["2019-12-31",]
#                      "2019-09-30","2019-06-30","2019-03-31","2018-12-31","2018-09-30","2018-06-30","2018-03-31","2017-12-31","2017-09-30","2017-06-30","2017-03-31","2016-12-31","2016-09-30","2016-06-30","2016-03-31","2015-12-31","2015-09-30","2015-06-30","2015-03-31","2014-12-31","2014-09-30","2014-06-30","2014-03-31","2013-12-31","2013-09-30","2013-06-30","2013-03-31","2012-12-31","2012-09-30","2012-06-30","2012-03-31","2011-12-31","2011-09-30","2011-06-30","2011-03-31","2010-12-31"]
    queue = []
    for semester in semester_list:
        offset = 0
        df = pd.DataFrame()
        while offset < 1237:
            url = url_genertor("point72+asset+management", offset, semester)
            
            # query processing...
            # df.to_csv('semester')
            offset += 40
    return queue
            
def url_genertor(fund:str, offset:int, semester:str)->str:
    prefix = "https://www.insidermonkey.com/hedge-fund"
    return f"{prefix}/{fund}/750/holdings/#/offset={offset}/&ffp={semester}&fot=3&fso=2"

print(len(get_data()))

0


In [4]:
header = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36",
    "sec-fetch-user": "?1",
    "dnt": "1",
    "accept-encoding": "gzip, deflate",
    "accept-language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7",
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "cookie": "_cmpQcif3pcsupported=1; _ga=GA1.2.1618055012.1591354476; _gid=GA1.2.386447438.1591354476; insidermonkey_auth=MIHLDBOtIUcORI6I3wXsuA%3D%3D",
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "same-origin",
    "upgrade-insecure-requests": "1",
}
url = 'https://www.insidermonkey.com/services/get_fund_holdings.php?hfid=750&module=holdings&fundType=0&ffp=2020-03-31&fot=3&fso=2&offset=40'

In [5]:
resp = requests.get(url=url, headers=header, verify=True, timeout=3)

In [6]:
soup = BeautifulSoup(resp.text, 'html.parser')

In [7]:
queue = []
need_modify = []

for idx, equity in enumerate(soup.find_all(class_=['even', 'odd'])):
    for element in equity(text=lambda text: isinstance(text, Comment)):
        element.extract()
    queue.append([])
    for item in equity:
        try:
            if item.get_text() == "":
                queue[idx].append(np.nan)
            else:
                queue[idx].append(item.get_text())
        except:
            continue

In [8]:
df = pd.DataFrame(queue, columns=['No.', 'Security', 'Ticker', 'Shares', 'Value (x$1000)', 'Activity(%)', 'Port(%)'])
df.loc[:, 'Security'] = df['Security'].replace(to_replace=r'PUTPUT', value=' PUT', regex=True)
df.loc[:, 'Security'] = df['Security'].replace(to_replace=r'CALLCALL', value=' CALL', regex=True)
# df.drop(columns=['No.'], inplace=True)

In [16]:
for item in equity:
    item.get_text()

AttributeError: 'NavigableString' object has no attribute 'get_text'

#### type processing 

In [173]:
df.loc[:,'No.'] = df['No.'].str.replace('.','').astype(np.int16)

for word, initial in {"$": "", ",": ""}.items():
    df['Value (x$1000)'] = df['Value (x$1000)'].str.replace(word, initial)
    
df['Value (x$1000)'] = df['Value (x$1000)'].astype(np.uint64)
# contain string
# df["Activity(%)"] 

df['Shares'] = df['Shares'].str.replace(',','').astype(int)

In [4]:
# mlc.save(pd.DataFrame(), 'x.feather')
df = mlc.load('point72-2020-03-31.feather')
df = pd.read_csv('point72-2020-03-31.csv')
df.to_csv('point72-2020-03-31.csv', index=False)

In [9]:
df = pd.read_csv('point72-2020-03-31.csv')

In [10]:
df.loc[:, 'Option'] = 0
df.loc[df['Security'].str.find("CALL", 1) != -1, 'Option'] = 1
df.loc[df['Security'].str.find("PUT", 1) != -1, 'Option'] = 2

In [13]:
df.loc[:, 'Security'] = df['Security'].replace({'CALL$': ' (CALL)', 'PUT$': ' (PUT)', 'CALLCALL$': ' (CALL)', 'PUTPUT$': ' (PUT)'}, regex=True)

In [27]:
col_opts = { 'editable': False }
col_defs = { 'Security': { 'editable': True } }
qgrid.show_grid(df, column_options=col_opts, column_definitions=col_defs)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

### explore null

### Fund ID

In [6]:
df_fund = pd.read_csv('Data/2020-06-09-05_fund_id.csv', index_col="Id")

In [14]:
for i in df_fund.to_dict()['Fund'].items():
    target_fund = i[1]
    fund_id = i[0]
    target_fund = target_fund.replace(' ', '_')
    print(target_fund, fund_id)

Berkshire_Hathaway 1
Soros_Fund_Management 2
BP_Capital 3
Icahn_Capital_LP 4
Renaissance_Technologies 5
Mohnish_Pabrai 6
Baupost_Group 7
SAC_Capital_Advisors 8
Navellier_&_Associates 9
Bridger_Management 10
Fisher_Asset_Management 11
Gotham_Asset_Management 12
Pershing_Square 13
ESL_Investments 14
Greenlight_Capital 15
Third_Point 16
Maverick_Capital 17
Paulson_&_Co 18
Lone_Pine_Capital 19
Dreman_Value_Management 20
Legg_Mason_Capital_Management 21
Tudor_Investment_Corp 23
Appaloosa_Management_LP 24
Tiger_Global_Management_LLC 25
Moore_Global_Investments 26
Capital_Growth_Management 27
Markel_Gayner_Asset_Management 28
Wallace_R._Weitz_&_Co. 29
Glenview_Capital 30
Viking_Global 31
Blue_Ridge_Capital 32
T2_Partners 33
Ariel_Investments 34
Senator_Investment_Group 35
Tiger_Consumer_Management 36
Eton_Park_Capital 37
Second_Curve_Capital 38
Axial_Capital 39
Carlson_Capital 40
Corsair_Capital_Management 41
Hound_Partners 42
Driehaus_Capital 43
Citadel_Investment_Group 44
Peconic_Partners_L

Pleasant_Lake_Partners 829
MAST_Capital 830
Permian_Investment_Partners 831
EcoR1_Capital 832
Alpenglow_Capital 833
Booth_Park_Capital_Management 834
Cloud_Gate_Capital 835
Shellback_Capital 836
Think_Investments 837
Craig_Drill_Capital 838
Discovery_Group 839
Lone_Star_Value_Management 840
Baker_Street_Capital 841
Limestone_Value_Partners 842
AlphaOne_Capital_Partners 843
Arlington_Value_Capital 844
Woodford_Investment_Management 845
Tiger_Pacific_Capital 846
Calixto_Global_Investors 847
Elmrox_Investment_Group 848
Alta_Park_Capital 849
Stillwater_Investment_Management 850
Frazier_Healthcare_Partners 851
Potrero_Capital_Research 852
Skybridge_Capital 853
Kavi_Asset_Management 854
Sabal_Capital_Management 855
Magnolia_Capital_Fund 856
Garelick_Capital_Partners 857
Deimos_Asset_Management 858
Immersion_Capital 859
Sand_Grove_Capital_Partners 860
Casdin_Capital 861
Asturias_Capital 862
Carbonado_Capital 863
Ashford_Investment_Management 864
Intermede_Investment_Partners 865
Valueworks_LL

In [3]:
df = pd.read_csv('/mnt/c/Users/morris/Downloads/13F/Data/2020-06-09-05_fund_id.csv', index_col="Id")
df.head()

,Fund,Manager
Id,,
1,Berkshire Hathaway,Warren Buffett
2,Soros Fund Management,George Soros
3,BP Capital,T Boone Pickens
4,Icahn Capital LP,Carl Icahn
5,Renaissance Technologies,Jim Simons


In [5]:
a = ['662','717','721','804','805','778','783','787','804','574','582','598','600','617','619','626','631','633','635','637','638','641','643','649','652','654','514','523','542','543','544','547','563','483','453','101','160','22']
df[df.index.isin(a)]

,Fund,Manager
Id,,
101,Adage Capital Management,Phill Gross And Robert Atchinson
160,Brookside Capital,Bain Capital
453,Dalton Investments,Gifford Combs
483,Jericho Capital Asset Management,Josh Resnick
514,Lomas Capital Management,Daniel Lascano
523,Crestwood Capital Management,Michael R. Weisberg
542,Red Mountain Capital,Willem Mesdag
543,Talara Capital Management,David Zusman
544,Springbok Capital,Gavin Saitowitz And Cisco J. Del Valle
